In [2]:
#Libraries
import cv2
import torch
%pip install -r requirements.txt 
from PIL import Image

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [3]:
import numpy as np
import os

def detect_license_plate(folder_path,  config_path, weights_path, confidence_threshold=0.5):
    # Load the YOLOv5 model
    model = torch.hub.load('C:\\Users\\Home\\Desktop\\ALPR system\\yolov5', 'custom', path=weights_path,source='local')
    model.yaml = config_path

    # Variables for accuracy calculation
    total_boxes = 0
    correct_boxes = 0

    # Iterate over images in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust the file extensions as needed
            image_path = os.path.join(folder_path, filename)

            # Load the input image
            img = Image.open(image_path)
            # Applying image preprocessing
            image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2GRAY)
            image = cv2.medianBlur(image, 5)  # Apply median blur
            image = cv2.GaussianBlur(image, (5, 5), 0)  # Apply Gaussian blur

            # Run license plate detection on the preprocessed image
            results = model(image)

            # Get the bounding box coordinates and confidence scores for the detected license plates
            pred = results.pred[0]
            boxes = pred[:, :4].detach().numpy()
            confidences = pred[:, 4].detach().numpy()

            # Iterate over the license plate detections
            for box, confidence in zip(boxes, confidences):
                if confidence > confidence_threshold:
                    # Draw the bounding box on the image
                    x_min, y_min, x_max, y_max = box.astype(int)
                    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    # Crop the license plate region
                    license_plate = img.crop((x_min, y_min, x_max, y_max))
                    # Write the confidence score on the license plate image
                    label = f"Confidence: {confidence:.2f}"
                    cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                    # Increase correct_boxes count
                    correct_boxes += 1

            # Increase total_boxes count
            total_boxes += len(boxes)

    # Calculate accuracy
    accuracy = correct_boxes / total_boxes if total_boxes > 0 else 0

    # Return the accuracy
    return accuracy


In [4]:
folder_path = 'C:\\Users\\Home\\Desktop\\ALPR system\\ALPR_GUI\\online dataset'
config_path = 'C:\\Users\\Home\\Desktop\\ALPR system\\yolov5\\data\\coco128.yaml'
weights_path = 'C:\\Users\\Home\\Desktop\\ALPR system\\best8.pt'
accuracy = detect_license_plate(folder_path, config_path,weights_path)

print("Accuracy:", accuracy)

YOLOv5  2023-6-2 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
folder_path = 'C:\\Users\\Home\\Desktop\\ALPR system\\ALPR_GUI\\testing imges'
config_path = 'C:\\Users\\Home\\Desktop\\ALPR system\\yolov5\\data\\coco128.yaml'
weights_path = 'C:\\Users\\Home\\Desktop\\ALPR system\\best8.pt'
accuracy = detect_license_plate(folder_path, config_path,weights_path)

print("Accuracy:", accuracy)

YOLOv5  2023-6-2 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Accuracy: 0.9514563106796117
